In [76]:
import pandas as pd

df1 = pd.read_csv('EA.csv', index_col=[0], sep=';')
df2 = pd.read_csv('REF.csv', index_col=[0], sep=';')

#first_column1 = df1.iloc[:, 0]
#first_column2 = df2.iloc[:, 0]

frames = [df1, df2]

result = pd.concat(frames)
result.T

Name,EA0,EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,REF0,REF1,REF2,REF3,REF4,REF5,REF6,REF7,REF8,REF9
Coord,"<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<437, 5, 200>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>","<3468, 5, 958>"
550,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1100,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1650,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2200,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2750,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3300,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
3850,5,5,5,5,5,5,5,5,5,5,4,4,4,4,4,4,4,4,4,4
4400,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
4950,7,8,7,7,7,6,7,7,7,7,6,6,6,6,6,6,6,6,6,5
